In [1]:
import requests
import re
import json
import pandas as pd
import datetime
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from collections import defaultdict
import html5lib


def get_dartOpenApi(code='005930', docs='사업,반기,분기', start_date='20010101', end_date=''):
    
    # 전자공시 오픈API 참조 : http://dart.fss.or.kr/dsap001/guide.do
    
    auth = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'             # 전자공시 오픈API 인증키
    headers={'Referer':'https://dart.fss.or.kr/dsap001/guide.do'} # 헤더: 실제 인증키를 사용시 사용하지 않아도 된다

    bsn_tps = ''
    dart_docs = []
    
    """ 아래 코드가 python스럽지만, 실행속도 측면에서는 그닥 ...
    if re.search('사업', docs): dart_docs.append('사업보고서')
    if re.search('반기', docs): dart_docs.append('반기보고서')   
    if re.search('분기', docs): dart_docs.append('분기보고서')   
    dart_bsn_tps = {'사업보고서' : 'A001', '반기보고서' : 'A002', '분기보고서' : 'A003'}
    for doc in dart_docs:
        bsn_tps += '&bsn_tp=' + dart_bsn_tps[doc]
    """
    if re.search('사업', docs): dart_docs.append('A001') 
    if re.search('반기', docs): dart_docs.append('A002')
    if re.search('분기', docs): dart_docs.append('A003') 
    if re.search('수시', docs): dart_docs.append('I001')     
    for i in range(0, len(dart_docs)):
        bsn_tps += '&bsn_tp=' + dart_docs[i]

    url_format = ('http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}' 
                    '&start_dt={start_date}&end_dt={end_date}&%s&page_set=100')
    url = url_format.format(auth=auth, code=code, start_date=start_date, end_date=end_date) % bsn_tps
    r = requests.get(url, headers=headers)
    jo = json.loads(r.text)
    result = json_normalize(jo, 'list')
    
    return result



def get_dartDocUrls(code='005930', docs='사업,반기,분기', start_date='20010101', end_date=''):
    
    df_api = get_dartOpenApi(code, docs, start_date, end_date)
    urls = []
    for i, rcpNo in df_api.iterrows():
        url_format = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'
        url = url_format.format(rcpNo['rcp_no'])
        urls.append(url)
    
    return urls
   
    
    
def get_dartDocTocUrls(head_url, toc_doc=''):
    
    # 공시보고서 목차(toc:table of contents) url들 리턴
    # toc_doc=목차제목 => 선택한 목차 url 리턴
    
    rcpNo = head_url[-14:]   # head_url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo:14자리}
    r = requests.get(head_url)

    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches: 
        params.append(m.replace("'", "").replace(" ", "").split(","))
   
    url_format = ('http://dart.fss.or.kr/report/viewer.do?rcpNo=%s'
                        '&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s')
    urls = []
    for p in params:
        if rcpNo == p[0]: urls.append(url_format % tuple(p))
            
    if toc_doc != '':
        if   re.search('회사', toc_doc): toc_num = 3
        elif re.search('사업', toc_doc): toc_num = 10
        elif re.search('재무', toc_doc): toc_num = 11
        elif re.search('감사인', toc_doc): toc_num = 18
        elif re.search('이사', toc_doc): toc_num = 19
        elif re.search('이사회', toc_doc): toc_num = 20
        elif re.search('주주', toc_doc): toc_num = 24
        elif re.search('임원', toc_doc): toc_num = 25
        elif re.search('계열회사', toc_doc): toc_num = 28
        elif re.search('이해관계자', toc_doc): toc_num = 29
        elif re.search('그 밖에', toc_doc): toc_num = 30
        elif re.search('전문가', toc_doc): toc_num = 31
        urls = urls[toc_num-1]
        
    return urls



def get_dartDocHtmlTable(doc_url, table_num=0):
    
    r = requests.get(doc_url)
    bs = BeautifulSoup(r.text, 'lxml')
    tables = bs.find_all('table')
    table = tables[table_num-1]
    df = pd.read_html(doc_url)
    
    return df



def get_htmlTable(table):
    
    """ ******************************************************************************************************
     get_htmlTable() : rowspan, colspan된 html table을 pandas dataframe으로 리턴
       - HTML table에 헤더가 없으면 아래 b.만 실행
       - 참조 코드 : https://stackoverflow.com/questions/9978445/parsing-a-table-with-rowspan-and-colspan   
    ==========================================================================================================
     입력인수 table은 html table, <table> ... </table>
     ----------------------------------------------------------------------------------------------------------
     a. 테이블 헤더에서, <theader>...<tr>...<th rowspan="rx" colspan="cy"> th_data </th>...</tr>...</theader>
        a1. rowspan된 개수(rx-1)만큼 아래쪽 행에 th_data 삽입
        a2. colspan된 개수(cy-1)만큼 오른쪽 열에 th_data 삽입
        a3. 테이블 헤더가 존재할 때, 헤더(th)에 rowspan, colspan이 있으면 헤더를 한개 행으로 병합, 상위 항목값은 [ ]로 병합
             -----------------------
            | X |     Y     |   Z   |       --------------------------------------
            |   |-------------------   =>  | X | A[Y] | B[Y] | C[Y] | D[Z] | E[Z] |
            |   | A | B | C | D | E |       --------------------------------------
             -----------------------
    ----------------------------------------------------------------------------------------------------------
     b. 테이블 바디에서, <tbody>...<tr>...<td rowspan="rx" colspan="cy"> td_data </td>...</tr>...</tbody>
        - rowspan된 개수(rx-1)만큼 아래쪽 행에 td_data 삽입
        - colspan된 개수(cy-1)만큼 오른쪽 열에 'NaN' 삽입
    ==========================================================================================================
     기타. 딕셔너리를 리스트로 하면 코딩이 편할 듯 한데, tbody_dct = defaultdict(list), 리스트 딕셔너리 ?
    ****************************************************************************************************** """
    
    thead = defaultdict(lambda : defaultdict(str)) 
    tbody = defaultdict(lambda : defaultdict(str))      
    for trow, tr in enumerate(table.find_all('tr')):
        # run a1. and a2.
        for tcol, th in enumerate(tr.find_all('th')):
            colspan = int(th.get('colspan', 1))             # colspan이 없으면, colspan = 1
            rowspan = int(th.get('rowspan', 1))             # rowspan이 없으면, rowspan = 1
            th_data = th.get_text(strip=True)               # 줄 바꿈('\n') 떼버림(strip)
            while trow in thead and tcol in thead[trow]:
                tcol += 1
            for i in range(trow, trow + rowspan):
                for j in range(tcol, tcol + colspan):       
                    thead[i][j] = th_data                   # rowspan, colspan된 th에 th_data 삽입
        # run b.
        for tcol, td in enumerate(tr.find_all('td')):
            colspan = int(td.get('colspan', 1))             # colspan이 없으면, colspan = 1
            rowspan = int(td.get('rowspan', 1))             # rowspan이 없으면, rowspan = 1
            td_data = td.get_text(strip=True)               # 줄 바꿈('\n') 떼버림(strip)
            while trow in tbody and tcol in tbody[trow]:
                tcol += 1
            for i in range(trow, trow + rowspan):
                tbody[i][tcol] = td_data                    # rowspan된 td에 td_data 삽입
                for j in range(tcol + 1, tcol + colspan):
                    tbody[i][j] = 'NaN'                     # colspan된 td에 NaN 삽입
    
    tbody = list(dict_to_listArray(tbody))                  # tbody(딕셔너리)를 tbdody(리스트 어레이)로 변환 

    # run a3.
    if table.find('thead') != None:  
        th_row = len(thead)-1
        for i in range(th_row-1, -1, -1):
            for j in range(0, len(thead[th_row])):
                if thead[th_row][j] != thead[i][j]:
                    thead[th_row][j] += '[' + thead[i][j] + ']'
        thead = thead[th_row].values()                      # thead(딕셔너리)의 마지막 element를 리스트로 변환
        df = pd.DataFrame(data = tbody, columns = thead)
    else:
        thead_list = []
        df = pd.DataFrame(data = tbody)
    
    return df

        
    
def dict_to_listArray(dict):
    
    for i, row in sorted(dict.items()):
        cols = []
        for j, col in sorted(row.items()):
            cols.append(col)
        yield cols      

        
        
def get_dartSpecificDocUrl(code='005930', doc='사업보고서', doc_date='2016.12'):
      
    date = doc_date.split('.')
    year = date[0]
    month = date[1]
    day = '30'

    # 보고서 3개월 이내 공시, 100일 후로 검색종료 일자 변경
    if month == '1Q' or month == '1q': month = '03'
    if month == '2Q' or month == '2q': month = '06'
    if month == '3Q' or month == '3q': month = '09'
    if month == '4Q' or month == '4q': month = '12'

    date = datetime.date(int(year), int(month), int(day))
    end_date = date + datetime.timedelta(days=100)
    end_date = str(end_date).replace('-','')
    start_date = year + month + day

    df_api = get_dartOpenApi(code, doc, start_date, end_date)

    if   re.search('사업', doc): doc_find = '사업보고서' + '\(' + year + '.' + month + '\)'
    elif re.search('반기', doc): doc_find = '반기보고서' + '\(' + year + '.' + month + '\)'
    elif re.search('분기', doc): doc_find = '분기보고서' + '\(' + year + '.' + month + '\)'

    for ix, rpt_nm in df_api.iterrows():
        if re.search(doc_find, rpt_nm['rpt_nm'].replace(' ','')):
            url_format = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'
            url = url_format.format(rpt_nm['rcp_no'])
            break
    
    return url



def get_dartSpecificDocHtmlTable(code='005930', doc='사업보고서', doc_date='2016.12', toc_doc='사업', table_num=1):

    doc_url = get_dartSpecificDocUrl(code, doc, doc_date)
    toc_url = get_dartDocTocUrls(doc_url, toc_doc)

    return get_dartDocHtmlTable(toc_url, table_num)

In [2]:
result = get_dartOpenApi(code='005930', docs='사업보고서', start_date='20100101', end_date='20170101')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005930,Y,삼성전자,삼성전자,20160330,20160330003536,연,사업보고서 (2015.12)
1,005930,Y,삼성전자,삼성전자,20150331,20150331002915,연,사업보고서 (2014.12)
2,005930,Y,삼성전자,삼성전자,20140331,20140331002427,연,사업보고서 (2013.12)
3,005930,Y,삼성전자,삼성전자,20130401,20130401003031,연,사업보고서 (2012.12)
4,005930,Y,삼성전자,삼성전자,20120330,20120330002110,연,[첨부추가]사업보고서 (2011.12)
5,005930,Y,삼성전자,삼성전자,20110331,20110331002193,연,[첨부추가]사업보고서 (2010.12)
6,005930,Y,삼성전자,삼성전자,20100331,20100331001680,연,[첨부추가]사업보고서 (2009.12)


In [3]:
result = get_dartDocUrls(code='005930', docs='사업보고서', start_date='20100101', end_date='20170101')
result[0]

'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330003536'

In [4]:
doc_urls = get_dartDocTocUrls(result[0])
doc_urls

['http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=1&offset=712&length=4053&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=2&offset=12770&length=410&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=3&offset=13184&length=232563&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=4&offset=13283&length=138879&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=5&offset=152166&length=20381&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=6&offset=172551&length=3614&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=7&offset=176169&length=54264&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=8&offset=230437&length=6828&dtd=dart3

In [5]:
doc_urls = get_dartDocTocUrls(result[0], toc_doc='사업')
doc_urls

'http://dart.fss.or.kr/report/viewer.do?rcpNo=20160330003536&dcmNo=5026126&eleId=10&offset=245751&length=217728&dtd=dart3.xsd'

In [11]:
df = get_dartDocHtmlTable(doc_urls, table_num=11)

#print(df)
df[13]

,부문,주요 제품,순매출액,비중,Unnamed: 4
0,CE 부문,"TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등",468954,23.4%,NaN
1,IM 부문,"HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등",1035543,51.6%,NaN
2,DS부문,반도체 사업,"DRAM, NAND Flash, 모바일AP 등",475868,23.7%
3,DP 사업,"TFT-LCD, OLED 등",274869,13.7%,NaN
4,부문 계,750261,37.4%,NaN,NaN
5,기타,-,"△248,223",△12.4%,NaN
6,전체 계,2006535,100.0%,NaN,NaN


In [7]:
# 위 일련 실행이, 아래 코드와 동일함
df = get_dartSpecificDocHtmlTable(code='005930', doc='사업보고서', doc_date='2016.12', toc_doc='사업', table_num=11)

print(df)
df

[        부문                           주요 제품                 Unnamed: 2
0    CE 부문  TV, 모니터, 냉장고, 세탁기, 에어컨, 의료기기 등                        NaN
1    IM 부문             HHP, 네트워크시스템, 컴퓨터 등                        NaN
2     DS부문                        반도체 사업부문  DRAM, NAND Flash, 모바일AP 등
3  DP 사업부문                 TFT-LCD, OLED 등                        NaN,   제 품  2016년  2015년  2014년
0  TV  21.6%  21.0%  22.6%,                                                    0
0  ※ 2014년, 2015년,2016년 시장점유율은 외부조사기관인 IHS의 세계시장점...,    제 품  2016년  2015년  2014년
0  HHP  19.2%  20.7%  22.4%,                                                    0
0  ※ 2014년, 2015년, 2016년 시장점유율은 외부조사기관인 Strategy ...,     제 품  2016년  2015년  2014년
0  DRAM  48.0%  45.3%  39.6%,                                                    0
0  ※ 2014년, 2015년, 2016년 시장점유율은 외부조사기관인 DRAMeXcha...,         제 품  2016년  2015년  2014년
0  디스플레이 패널  17.1%  21.1%  20.9%,                                                    0
0  ※ 2014년, 2015년, 2016년 시장점유율은 외부조사

[        부문                           주요 제품                 Unnamed: 2
 0    CE 부문  TV, 모니터, 냉장고, 세탁기, 에어컨, 의료기기 등                        NaN
 1    IM 부문             HHP, 네트워크시스템, 컴퓨터 등                        NaN
 2     DS부문                        반도체 사업부문  DRAM, NAND Flash, 모바일AP 등
 3  DP 사업부문                 TFT-LCD, OLED 등                        NaN,
   제 품  2016년  2015년  2014년
 0  TV  21.6%  21.0%  22.6%,
                                                    0
 0  ※ 2014년, 2015년,2016년 시장점유율은 외부조사기관인 IHS의 세계시장점...,
    제 품  2016년  2015년  2014년
 0  HHP  19.2%  20.7%  22.4%,
                                                    0
 0  ※ 2014년, 2015년, 2016년 시장점유율은 외부조사기관인 Strategy ...,
     제 품  2016년  2015년  2014년
 0  DRAM  48.0%  45.3%  39.6%,
                                                    0
 0  ※ 2014년, 2015년, 2016년 시장점유율은 외부조사기관인 DRAMeXcha...,
         제 품  2016년  2015년  2014년
 0  디스플레이 패널  17.1%  21.1%  20.9%,
                                                    0
 0  ※ 2014년, 2015